In [1]:
import os
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from transformers import RobertaTokenizer, RobertaModel
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski, Fragments, rdMolDescriptors
import deepchem as dc

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/HDD1/bbq9088/miniconda3/envs/molberta/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [2]:
# Move model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [3]:
# Load RoBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained("./origin_model/roberta/tokenizer_folder")
model = RobertaModel.from_pretrained("./origin_model/roberta").to(device)

/HDD1/bbq9088/miniconda3/envs/molberta/lib/python3.10/site-packages/transformers/modeling_utils.py:442: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(check

In [4]:
# CircularFingerprint for dataset featurization
featurizer = dc.feat.CircularFingerprint(radius=2, size=2048)

In [5]:
# Load datasets
tasks_freesolv, datasets_freesolv, _ = dc.molnet.load_freesolv(featurizer=featurizer, splitter=None, transformers=[], reload=True)
dataset_freesolv = datasets_freesolv[0]
df_freesolv = pd.DataFrame({'smiles': dataset_freesolv.ids, 'label': dataset_freesolv.y[:, 0]}).dropna()

tasks_lipo, datasets_lipo, _ = dc.molnet.load_lipo(featurizer=featurizer, splitter=None, transformers=[], reload=True)
dataset_lipo = datasets_lipo[0]
df_lipo = pd.DataFrame({'smiles': dataset_lipo.ids, 'label': dataset_lipo.y[:, 0]}).dropna()

tasks_esol, datasets_esol, _ = dc.molnet.load_delaney(featurizer=featurizer, splitter=None, transformers=[], reload=True)
dataset_esol = datasets_esol[0]
df_esol = pd.DataFrame({'smiles': dataset_esol.ids, 'label': dataset_esol.y[:, 0]}).dropna()

In [6]:
# Function to calculate molecular properties
def calculate_properties(smiles):
    mol = Chem.MolFromSmiles(smiles)
    properties = []
    try:
        properties.append(Descriptors.MolWt(mol) if mol else None)
        properties.append(Crippen.MolLogP(mol) if mol else None)
        properties.append(Descriptors.TPSA(mol) if mol else None)
        properties.append(Lipinski.NumHAcceptors(mol) if mol else None)
        properties.append(Lipinski.NumHDonors(mol) if mol else None)
        properties.append(Lipinski.NumRotatableBonds(mol) if mol else None)
        properties.append(Chem.GetFormalCharge(mol) if mol else None)
        properties.append(rdMolDescriptors.CalcNumAtomStereoCenters(mol) if mol else None)
        properties.append(rdMolDescriptors.CalcFractionCSP3(mol) if mol else None)
        properties.append(Descriptors.NumAliphaticCarbocycles(mol) if mol else None)
        properties.append(Descriptors.NumAromaticRings(mol) if mol else None)
        properties.append(Descriptors.NumHeteroatoms(mol) if mol else None)
        properties.append(Fragments.fr_COO(mol) if mol else None)
        properties.append(Fragments.fr_Al_OH(mol) if mol else None)
        properties.append(Fragments.fr_alkyl_halide(mol) if mol else None)
        properties.append(Descriptors.NumAromaticCarbocycles(mol) if mol else None)
        properties.append(Fragments.fr_piperdine(mol) if mol else None)
        properties.append(Fragments.fr_methoxy(mol) if mol else None)
    except Exception as e:
        print(f"Warning: Could not calculate properties for SMILES: {smiles}. Error: {e}")
        return [None] * 18
    return properties

In [7]:
# Create input text for SMILES
def create_input_text(smiles):
    properties = calculate_properties(smiles)
    property_names = [
        "Molecular Weight", "LogP", "Topological Polar Surface Area",
        "Number of Hydrogen Bond Acceptors", "Number of Hydrogen Bond Donors",
        "Number of Rotatable Bonds", "Formal Charge", "Number of Atom Stereocenters",
        "Fraction of sp3 Carbon Atoms", "Number of Aliphatic Carbocycles",
        "Number of Aromatic Rings", "Number of Heteroatoms", "Number of Carboxylic Acid Groups",
        "Number of Aliphatic Alcohol Groups", "Number of Alkyl Halide Groups",
        "Number of Aromatic Carbocycles", "Number of Piperidine Groups",
        "Number of Methoxy Groups"
    ]
    properties_text = " | ".join(
        [f"{name}: {value:.5f}" if value is not None else f"{name}: None"
         for name, value in zip(property_names, properties)]
    )
    input_text = f"SMILES: {smiles} | {properties_text}"
    return input_text

In [8]:
# Function to get embeddings using RoBERTa
def get_embeddings(model, tokenizer, smiles_list):
    embeddings = []
    for smiles in tqdm(smiles_list, desc="Processing SMILES with Roberta"):
        input_text = create_input_text(smiles)
        inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).cpu().numpy().flatten()
        embeddings.append(embedding)
    return np.array(embeddings)

In [9]:
# Ridge regression with K-Fold cross-validation
def train_and_evaluate_regression_kfold(X, y, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    rmse_scores = []
    r2_scores = []

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model = Ridge()
        model.fit(X_train, y_train.ravel())
        y_pred = model.predict(X_test)

        # Z-score standardization
        y_test_mean, y_test_std = np.mean(y_test), np.std(y_test)
        y_test_z = (y_test - y_test_mean) / y_test_std
        y_pred_z = (y_pred - y_test_mean) / y_test_std

        # Metrics
        rmse_scores.append(np.sqrt(mean_squared_error(y_test_z, y_pred_z)))
        r2_scores.append(r2_score(y_test_z, y_pred_z))

    print(f"Average RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}")
    print(f"Average R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}")
    return rmse_scores, r2_scores

In [10]:
# Dataset processing and evaluation
embedding_dir = os.path.join(os.getcwd(), 'Embedding')
result_dir = os.path.join(os.getcwd(), 'Result/Properties')
os.makedirs(embedding_dir, exist_ok=True)
os.makedirs(result_dir, exist_ok=True)

In [11]:
for dataset_name, df in [("FreeSolv", df_freesolv), ("Lipophilicity", df_lipo), ("ESOL", df_esol)]:
    print(f"\n=== Processing {dataset_name} ===")

    # SMILES embedding
    embeddings = get_embeddings(model, tokenizer, df['smiles'].tolist())
    embedding_file = os.path.join(embedding_dir, f"{dataset_name}_roberta_embeddings.csv")
    pd.DataFrame(embeddings).to_csv(embedding_file, index=False)
    print(f"Saved embeddings to {embedding_file}")

    # Regression evaluation
    labels = df['label'].values
    rmse_scores, r2_scores = train_and_evaluate_regression_kfold(embeddings, labels)

    # Save results
    result_file = os.path.join(result_dir, f"{dataset_name}_regression_results.txt")
    with open(result_file, 'w') as f:
        f.write(f"Dataset: {dataset_name}\n")
        f.write(f"Average RMSE: {np.mean(rmse_scores):.4f} ± {np.std(rmse_scores):.4f}\n")
        f.write(f"Average R²: {np.mean(r2_scores):.4f} ± {np.std(r2_scores):.4f}\n")

    print(f"Results for {dataset_name} saved to {result_file}")


=== Processing FreeSolv ===


Processing SMILES with Roberta: 100%|████████████████████████████████████████████| 642/642 [00:06<00:00, 99.52it/s]


Saved embeddings to /HDD1/bbq9088/GPT-MolBERTa/model_predict/Embedding/FreeSolv_roberta_embeddings.csv
Average RMSE: 0.4428 ± 0.0453
Average R²: 0.8019 ± 0.0403
Results for FreeSolv saved to /HDD1/bbq9088/GPT-MolBERTa/model_predict/Result/Properties/FreeSolv_regression_results.txt

=== Processing Lipophilicity ===


Processing SMILES with Roberta: 100%|██████████████████████████████████████████| 4200/4200 [00:43<00:00, 97.66it/s]


Saved embeddings to /HDD1/bbq9088/GPT-MolBERTa/model_predict/Embedding/Lipophilicity_roberta_embeddings.csv
Average RMSE: 0.7820 ± 0.0067
Average R²: 0.3885 ± 0.0105
Results for Lipophilicity saved to /HDD1/bbq9088/GPT-MolBERTa/model_predict/Result/Properties/Lipophilicity_regression_results.txt

=== Processing ESOL ===


Processing SMILES with Roberta: 100%|█████████████████████████████████████████| 1128/1128 [00:11<00:00, 102.34it/s]


Saved embeddings to /HDD1/bbq9088/GPT-MolBERTa/model_predict/Embedding/ESOL_roberta_embeddings.csv
Average RMSE: 0.4905 ± 0.0365
Average R²: 0.7580 ± 0.0355
Results for ESOL saved to /HDD1/bbq9088/GPT-MolBERTa/model_predict/Result/Properties/ESOL_regression_results.txt
